In [68]:
import pickle
with open('cleaned_strings', 'rb') as f:
    data = pickle.load(f)

In [69]:
data

['slow moving aimless movie distressed drifting young man',
 'not sure lost flat characters audience nearly half walked',
 'attempting artiness black white clever camera angles movie disappointed became even ridiculous acting poor plot lines almost non existent',
 'little music anything speak',
 'best scene movie gerardo trying find song keeps running head',
 'rest movie lacks art charm meaning emptiness works guess empty',
 'wasted two hours',
 'saw movie today thought good effort good messages kids',
 'bit predictable',
 'loved casting jimmy buffet science teacher',
 'baby owls adorable',
 'movie showed lot florida best made look appealing',
 'songs best muppets hilarious',
 'cool',
 'right case movie delivers everything almost right face',
 'average acting main person low budget clearly see',
 'review long overdue since consider tale two sisters single greatest film ever made',
 'put gem movie terms screenplay cinematography acting post production editing directing aspect film makin

In [70]:
# Function to calculate the idf value of all the words in corpus.

def idf_values(word, all_docs): 
    num_docs_with_given_word = 0
    for i in range(len(all_docs)): 
        if word.lower() in all_docs[i].lower().split(): 
            num_docs_with_given_word += 1
  
    if num_docs_with_given_word > 0: 
        # Total number of documents 
        total_num_docs = len(all_docs)  
  
        # Calculating the IDF  
        idf_val = log(float(total_num_docs) / num_docs_with_given_word) 
        return idf_val  
    else: 
        return 0   

In [71]:
from tqdm import tqdm # tqdm is a library that helps us to visualize the runtime of for loop
# https://tqdm.github.io/
from math import log
from collections import OrderedDict, Counter
from operator import itemgetter
from scipy.sparse import csr_matrix
dic = {}
idf = {}
idfvalue = []  # will accept list of sentences

def fit(dataset):    
    unique_words = set() 
    # check if its list type or not
    if isinstance(dataset, (list,)):
        for row in dataset: 
            for word in row.split(" "): # converts a string into list of words
                if len(word) < 2:       # neglecting words of length less than 2
                    continue
                else:
                    unique_words.add(word)
        unique_words = sorted(list(unique_words))
        print(unique_words)
        vocab = {j:i for i,j in enumerate(unique_words)}
        # find the count of each word in corpus
        for i in vocab:
            count = 0
            for j in range(0,len(dataset)):
                x = dataset[j].find(i)     # find the word in each dataset
                if x > -1:
                    count+=1
            dic[i] = count
        # order the dictionary in ascending
        if len(idfvalue) == 0:
            for i in range(len(unique_words)):
                idfvalue.append(idf_values(unique_words[i],data))
        for i in range(len(unique_words)):
            idf[unique_words[i]] = idfvalue[i]
        return dic,vocab,idfvalue,idf
    else:
        print("Pass a list of sentence")

In [72]:
fit(data)[3]

['aailiyah', 'abandoned', 'ability', 'abroad', 'absolutely', 'abstruse', 'abysmal', 'academy', 'accents', 'accessible', 'acclaimed', 'accolades', 'accurate', 'accurately', 'accused', 'achievement', 'achille', 'ackerman', 'act', 'acted', 'acting', 'action', 'actions', 'actor', 'actors', 'actress', 'actresses', 'actually', 'adams', 'adaptation', 'add', 'added', 'addition', 'admins', 'admiration', 'admitted', 'adorable', 'adrift', 'adventure', 'advise', 'aerial', 'aesthetically', 'affected', 'affleck', 'afraid', 'africa', 'afternoon', 'age', 'aged', 'ages', 'ago', 'agree', 'agreed', 'aimless', 'air', 'aired', 'akasha', 'akin', 'alert', 'alexander', 'alike', 'allison', 'allow', 'allowing', 'almost', 'along', 'alongside', 'already', 'also', 'although', 'always', 'amateurish', 'amaze', 'amazed', 'amazing', 'amazingly', 'america', 'american', 'americans', 'among', 'amount', 'amusing', 'amust', 'anatomist', 'angel', 'angela', 'angeles', 'angelina', 'angle', 'angles', 'angry', 'anguish', 'angus

{'aailiyah': 6.61472560020376,
 'abandoned': 6.61472560020376,
 'ability': 5.516113311535651,
 'abroad': 6.61472560020376,
 'absolutely': 4.417501022867541,
 'abstruse': 6.61472560020376,
 'abysmal': 5.921578419643816,
 'academy': 6.61472560020376,
 'accents': 6.61472560020376,
 'accessible': 6.61472560020376,
 'acclaimed': 6.61472560020376,
 'accolades': 6.61472560020376,
 'accurate': 6.61472560020376,
 'accurately': 6.61472560020376,
 'accused': 5.921578419643816,
 'achievement': 5.921578419643816,
 'achille': 6.61472560020376,
 'ackerman': 6.61472560020376,
 'act': 5.921578419643816,
 'acted': 5.516113311535651,
 'acting': 3.0038076875595365,
 'action': 4.668815451148448,
 'actions': 6.61472560020376,
 'actor': 4.417501022867541,
 'actors': 3.724353842307596,
 'actress': 5.516113311535651,
 'actresses': 5.516113311535651,
 'actually': 4.312140507209715,
 'adams': 6.61472560020376,
 'adaptation': 5.921578419643816,
 'add': 6.61472560020376,
 'added': 6.61472560020376,
 'addition': 5.

In [79]:
def transform(dataset,vocab, wrd_in_doc):
    rows = []
    columns = []
    values = []
    if isinstance(dataset, (list,)):
        for idx, row in enumerate(tqdm(dataset)): 
            # {word:frequency}
            word_freq = dict(Counter(row.split()))
            # for every unique word in the document
            for word, freq in word_freq.items():               
                if len(word) < 2:
                    continue
                # we will check if its there in the vocabulary that we build in fit() function
                # dict.get() function will return the values, if the key doesn't exits it will return -1
                col_index = vocab.get(word, -1) # retreving the dimension number of a word()

                if col_index !=-1:
                    rows.append(idx)
                    columns.append(col_index)
                    # applying the formula of tf-idf in value
                    values.append(freq/len(row.split())*(log(len(dataset)/wrd_in_doc.get(word))))
        return csr_matrix((values, (rows,columns)), shape=(len(dataset),len(vocab)))
    else:
        print("Pass list of strings")

In [80]:
uniqueword = fit(data)[1] 
word_in_doc = fit(data)[0]

['aailiyah', 'abandoned', 'ability', 'abroad', 'absolutely', 'abstruse', 'abysmal', 'academy', 'accents', 'accessible', 'acclaimed', 'accolades', 'accurate', 'accurately', 'accused', 'achievement', 'achille', 'ackerman', 'act', 'acted', 'acting', 'action', 'actions', 'actor', 'actors', 'actress', 'actresses', 'actually', 'adams', 'adaptation', 'add', 'added', 'addition', 'admins', 'admiration', 'admitted', 'adorable', 'adrift', 'adventure', 'advise', 'aerial', 'aesthetically', 'affected', 'affleck', 'afraid', 'africa', 'afternoon', 'age', 'aged', 'ages', 'ago', 'agree', 'agreed', 'aimless', 'air', 'aired', 'akasha', 'akin', 'alert', 'alexander', 'alike', 'allison', 'allow', 'allowing', 'almost', 'along', 'alongside', 'already', 'also', 'although', 'always', 'amateurish', 'amaze', 'amazed', 'amazing', 'amazingly', 'america', 'american', 'americans', 'among', 'amount', 'amusing', 'amust', 'anatomist', 'angel', 'angela', 'angeles', 'angelina', 'angle', 'angles', 'angry', 'anguish', 'angus

['aailiyah', 'abandoned', 'ability', 'abroad', 'absolutely', 'abstruse', 'abysmal', 'academy', 'accents', 'accessible', 'acclaimed', 'accolades', 'accurate', 'accurately', 'accused', 'achievement', 'achille', 'ackerman', 'act', 'acted', 'acting', 'action', 'actions', 'actor', 'actors', 'actress', 'actresses', 'actually', 'adams', 'adaptation', 'add', 'added', 'addition', 'admins', 'admiration', 'admitted', 'adorable', 'adrift', 'adventure', 'advise', 'aerial', 'aesthetically', 'affected', 'affleck', 'afraid', 'africa', 'afternoon', 'age', 'aged', 'ages', 'ago', 'agree', 'agreed', 'aimless', 'air', 'aired', 'akasha', 'akin', 'alert', 'alexander', 'alike', 'allison', 'allow', 'allowing', 'almost', 'along', 'alongside', 'already', 'also', 'although', 'always', 'amateurish', 'amaze', 'amazed', 'amazing', 'amazingly', 'america', 'american', 'americans', 'among', 'amount', 'amusing', 'amust', 'anatomist', 'angel', 'angela', 'angeles', 'angelina', 'angle', 'angles', 'angry', 'anguish', 'angus

In [81]:
print(transform(data, uniqueword, word_in_doc)[0].toarray())

100%|█████████████████████████████████████████████████████████████████████████████| 746/746 [00:00<00:00, 17618.16it/s]

[[0. 0. 0. ... 0. 0. 0.]]
